In [31]:
import pandas as pd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
from sklearn.model_selection import train_test_split

In [32]:
def scan_size(path_im):
    images = {}
    for path in path_im[:500]:
        # récupération nom de l'image
        im_name = os.path.basename(path)
        
        # récupération des images
        img = cv2.imread(path)
        images[int(im_name[:-4])] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
    return images

In [33]:
im_path = glob.glob('../../../images_training_rev1_resize_black/*.jpg')

In [34]:
images = scan_size(im_path)

In [35]:
images_id = list(images.keys())

In [36]:
Y_trainfull = pd.read_csv('./csv/max_benchmark.csv', index_col=0)

In [37]:
Y_trainfull = Y_trainfull.loc[images_id, :]
Y_trainfull

,shape_Class1.1,shape_Class1.2,shape_Class1.3
GalaxyID,,,
100008,0,1,0
100023,0,1,0
100078,1,0,0
100090,1,0,0
100122,1,0,0
...,...,...,...
107831,0,1,0
107849,1,0,0
107859,0,1,0


In [38]:
X_train_id, x_val_id, Y_train, y_val = train_test_split(images_id, Y_trainfull, test_size=0.2, random_state=15)

In [39]:
print(len(X_train))
print(len(Y_train))
print(len(x_val))
print(len(y_val))

400
400
100
100


In [40]:
def images_from_id(images_id, images):
    images_selected = []
    
    for image_id in images_id:
        images_selected += [images[image_id]]
    
    return images_selected

In [41]:
X_train = images_from_id(X_train_id, images)
x_val = images_from_id(x_val_id, images)

In [42]:
print(len(X_train))
X_train[0].shape

400


(66, 66, 3)